About the Dataset:
<>
sample_data train.csv
train.csv (94.06M)|
1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:

importing the dependencies


In [3]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [7]:
#loading dataset to pandas dataframe - its makes data more structured
news_dataset = pd.read_csv('/content/train.csv')

In [10]:
news_dataset.shape


(20800, 5)

In [9]:
#print first 5 rows
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [11]:
#counting the no of missing values in dataset
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [13]:
#replacing the null value with empty string
news_dataset = news_dataset.fillna('')

In [14]:
#merging the author name and news tittle
news_dataset['content'] = news_dataset['author']+news_dataset['title']

In [15]:
print(news_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [16]:
#seprating the data and label
x=news_dataset.drop(columns='label',axis=1)
y=news_dataset['label']

In [17]:
print(x)
print(y)

          id  ...                                            content
0          0  ...  Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1          1  ...  Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2          2  ...  Consortiumnews.comWhy the Truth Might Get You ...
3          3  ...  Jessica Purkiss15 Civilians Killed In Single U...
4          4  ...  Howard PortnoyIranian woman jailed for fiction...
...      ...  ...                                                ...
20795  20795  ...  Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796  20796  ...  Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797  20797  ...  Michael J. de la Merced and Rachel AbramsMacy’...
20798  20798  ...  Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799  20799  ...             David SwansonWhat Keeps the F-35 Alive

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

In [18]:
#stemming is a process to reduce a word to its root word
port_stem = PorterStemmer()


In [19]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]','',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [21]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [22]:
print(news_dataset['content'])

0        darrelllucushousedemaidewedidntevenseecomeysle...
1        danieljflynnflynnhillaryclintonbigwomanoncampu...
2               consortiumnewscomwhythetruthmightgetyoufir
3        jessicapurkisscivilianskilledinsingleusairstri...
4        howardportnoyiranianwomanjailedforfictionalunp...
                               ...                        
20795    jeromehudsonrappertitrumpaposterchildforwhites...
20796    benjaminhoffmannflplayoffsschedulematchupsando...
20797    michaeljdelamercedandrachelabramsmacysissaidto...
20798    alexansarynatorussiatoholdparallelexercisesinb...
20799                            davidswansonwhatkeepsthef
Name: content, Length: 20800, dtype: object


In [23]:
#seprating the data and label
x=news_dataset['content'].values
y=news_dataset['label'].values

In [24]:
print(x)

['darrelllucushousedemaidewedidntevenseecomeysletteruntiljasonchaffetztweetedit'
 'danieljflynnflynnhillaryclintonbigwomanoncampusbreitbart'
 'consortiumnewscomwhythetruthmightgetyoufir' ...
 'michaeljdelamercedandrachelabramsmacysissaidtoreceivetakeoverapproachbyhudsonsbaythenewyorktim'
 'alexansarynatorussiatoholdparallelexercisesinbalkan'
 'davidswansonwhatkeepsthef']


In [25]:
print(y)

[1 0 1 ... 0 1 1]


In [26]:
y.shape

(20800,)

In [27]:
#coverting all the text into num as computer can understand only num not text
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [28]:
print(x)

  (0, 5208)	1.0
  (1, 4896)	1.0
  (2, 4549)	1.0
  (3, 9741)	1.0
  (4, 7968)	1.0
  (5, 4994)	1.0
  (6, 11988)	1.0
  (7, 1010)	1.0
  (8, 6803)	1.0
  (9, 12883)	1.0
  (10, 85)	1.0
  (11, 4201)	1.0
  (12, 1165)	1.0
  (13, 9095)	1.0
  (14, 1698)	1.0
  (15, 8725)	1.0
  (16, 13016)	1.0
  (17, 17437)	1.0
  (18, 18102)	1.0
  (19, 9269)	1.0
  (20, 14162)	1.0
  (21, 9651)	1.0
  (22, 15084)	1.0
  (23, 12692)	1.0
  (24, 5799)	1.0
  :	:
  (20775, 1824)	1.0
  (20776, 8919)	1.0
  (20777, 408)	1.0
  (20778, 13973)	1.0
  (20779, 18501)	1.0
  (20780, 4165)	1.0
  (20781, 7747)	1.0
  (20782, 18134)	1.0
  (20783, 3726)	1.0
  (20784, 5551)	1.0
  (20785, 1573)	1.0
  (20786, 7464)	1.0
  (20787, 8238)	1.0
  (20788, 9851)	1.0
  (20789, 19648)	1.0
  (20790, 288)	1.0
  (20791, 4863)	1.0
  (20792, 10228)	1.0
  (20793, 16406)	1.0
  (20794, 11851)	1.0
  (20795, 9666)	1.0
  (20796, 2366)	1.0
  (20797, 13127)	1.0
  (20798, 890)	1.0
  (20799, 5474)	1.0


In [29]:
#splitting the dataset to training & test data
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size =0.2,stratify=y,random_state=2)

In [30]:
#training logistic regresion model
model = LogisticRegression()

In [33]:
model.fit(X_train,Y_train)

LogisticRegression()

In [34]:
#evaluation
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
trainin_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [35]:
print('accuracy score of training data:',trainin_data_accuracy)

accuracy score of training data: 0.9927283653846154


In [36]:
#evaluation
#accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [38]:
print('accuracy score of test data:',test_data_accuracy)

accuracy score of test data: 0.5329326923076924


In [52]:
#making predictive system
X_new = X_test[2]
prediction = model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('the news is real')
else:
  print('the news is fake')

[0]
the news is real


In [53]:
print(Y_test[2])

1
